# Benchmark modelos de Fasttext (BIO) para obtener Embeddings del texto




In [ ]:
! python -m spacy download es_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-27 09:40:09.747809: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
! pip install spacy
#! python -m spacy download es
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def NLP_preprocesaDocumento(nlp, doc):

  #Convertir a minúsculas
  doc = doc.lower()
  
  #Tokenizado
  doc = [" ".join([token.text for token in nlp(doc)])][0]

  #Eliminación de signos de puntuación
  doc = [" ".join([token.lemma_ for token in nlp(doc) if not token.is_punct]) ][0]

  #Eliminación de stopwords
  spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
  doc = [" ".join([token for token in doc.split() if not token.lower() in spacy_stopwords])][0]
  
  #Lematización de palabras
  doc = [" ".join([token.lemma_ for token in nlp(doc)])][0]

  #Eliminación de tildes
  tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'ü': 'u', 'Ü': 'U'}
  for tilde in tildes:
	  if tilde in doc:
		  doc = doc.replace(tilde, tildes[tilde])

  return doc


In [ ]:
def split_sentences(nlp, text):
    tokens = nlp(text)
    sentences = []
    for sent in tokens.sents:
        sentences.append(sent.text.strip())
    return sentences

In [ ]:
#Cargamos el modelo en español
nlp_lg = spacy.load("es_core_news_lg")

## Carga de las terminologías clínicas SNOMED-CT y generación de sus Embeddings
NOTA: Estos solo es necesario hacerlo la primera vez

In [ ]:
import os
def cargaProblemasSaludAHSpainExtension():
  f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
  terminos_snomedct_problemassaludAH = []
  cont = 0
  for row in f:
    if cont>0:
      values = row.strip().split("\t")
      #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
      #print(values)
      termino = {}
      termino["id"] = values[0];
      #termino["effectiveTime"] = values[1];
      #termino["active"] = values[2];
      #termino["moduleId"] = values[3];
      #termino["refsetId"] = values[4];
      #termino["referencedComponentId"] = values[5];
      termino["term"] = values[6];
      #termino["num_palabras"] = len(values[6].split(" "));
      #print(termino)
      terminos_snomedct_problemassaludAH.append(termino)
    cont = cont + 1

  f.close();
  return terminos_snomedct_problemassaludAH;

In [ ]:
def anadirEmbedding(model,nlp,lista,num_items_max=0,preprocesado=False):
  cont = 0
  for item in lista:
    #print(item["term"])
    if preprocesado:
      doc = NLP_preprocesaDocumento(nlp,item["term"])
      if doc == "":
        print("ERROR AL Parsear " + item["term"])
        doc = item["term"]
    else:
      doc = item["term"]
    #print(doc)
    item["embedding"] = model.wv.get_vector(doc)
    cont = cont + 1
    if cont == num_items_max:
      break
  return lista

In [ ]:
terminos_snomedct_problemassaludAH = cargaProblemasSaludAHSpainExtension();

In [ ]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

#numpyArrayOne = numpy.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]])

# Serialization
#numpyData = {"array": numpyArrayOne}
#encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
#print("Printing JSON serialized NumPy array")
#print(encodedNumpyData)

# Deserialization
#print("Decode JSON serialized NumPy array")
#decodedArrays = json.loads(encodedNumpyData)

#finalNumpyArray = numpy.asarray(decodedArrays["array"])
#print("NumPy Array")
#print(finalNumpyArray)

In [ ]:
!pip install gensim
from gensim.models import FastText
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Obtención de embeddings con el modelo seleccionado (nlp_lg y model sp-cli-w-emb-uncased-cbow-d50-clinic_es)

In [ ]:
#Tiempo de ejecucion: 60 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d50-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 60 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-w-emb-uncased-cbow-d50-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 120 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d100-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 60 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-w-emb-uncased-cbow-d100-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 180 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d300-bio_es.bin')

In [ ]:
#Tiempo de ejecucion: 60 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);

ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)

In [ ]:
with open('catalogo_sp-bio-w-emb-uncased-cbow-d300-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 60 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d50-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 48 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-w-emb-uncased-skip-d50-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 60 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d100-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 48 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-w-emb-uncased-skip-d100-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

In [ ]:
#Tiempo de ejecucion: 120 sg
model = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d300-bio_es.bin')


In [ ]:
#Tiempo de ejecucion: 60 sg
terminos_snomedct_problemassaludAH = anadirEmbedding(model, nlp_lg, terminos_snomedct_problemassaludAH, 0, True);


ERROR AL Parsear estado de gran mal


In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = json.dumps(termino["embedding"], cls=NumpyArrayEncoder)
with open('catalogo_sp-bio-w-emb-uncased-skip-d300-bio_es.json', 'w') as file:
    json.dump(terminos_snomedct_problemassaludAH, file)
file.close()

## Obtención de embeddings, carga de modelos y pruebas

In [ ]:
!pip install gensim
from gensim.models import FastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def obtenEmbedding(model, doc):
  return model.wv.get_vector(doc)

In [ ]:
model_cc50 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d50-bio_es.bin')
model_cc100 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d100-bio_es.bin')
model_cc300 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-cbow-d300-bio_es.bin')


In [ ]:
model_cs50 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d50-bio_es.bin')
model_cs100 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d100-bio_es.bin')
#model_cs300 = FastText.load_fasttext_format('drive/MyDrive/models/sp-bio-w-emb-uncased-skip-d300-bio_es.bin')

In [ ]:
import os, json, numpy

f = open('catalogo_sp-bio-w-emb-uncased-cbow-d50-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc50 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc50.append(termino)
f.close();

f = open('catalogo_sp-bio-w-emb-uncased-cbow-d100-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc100 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc100.append(termino)
f.close();

f = open('catalogo_sp-bio-w-emb-uncased-cbow-d300-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cc300 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cc300.append(termino)
f.close();

In [ ]:
f = open('catalogo_sp-bio-w-emb-uncased-skip-d50-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs50 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs50.append(termino)
f.close();

f = open('catalogo_sp-bio-w-emb-uncased-skip-d100-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs100 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs100.append(termino)
f.close();

f = open('catalogo_sp-bio-w-emb-uncased-skip-d300-bio_es.json')
data = json.load(f)
terminos_snomedct_problemassaludAH_cs300 = []
for termino in data:
  termino["embedding"] = numpy.asarray(json.loads(termino["embedding"]));
  terminos_snomedct_problemassaludAH_cs300.append(termino)
f.close();

In [ ]:
def NLP_get_ngrams(text, n):
    text = text.split()
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

In [ ]:
#informe = "El paciente presenta el abdomen distenido ."
#informe = "El paciente presenta un aborto espontáneo y retenido, distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
#informe = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."


In [ ]:
import numpy as np
from numpy.linalg import norm
from operator import itemgetter

def get_terminos_vinculados(model, catalog, similarity_threshold, similarity, informe):
  terminos_vinculados = []

  frases = split_sentences(nlp_lg, informe)
  for frase in frases:
    informe_preprocesado = NLP_preprocesaDocumento(nlp_lg, frase)

    #Empezamos por ngramas más grandes a más pequeños
    for num_ngrams in range(6, 0, -1):
      ngramas = NLP_get_ngrams(informe_preprocesado, num_ngrams)
      for ngrama in ngramas:
        doc = " ".join([token for token in ngrama])
        #print(doc)
        doc_embedding = model.wv.get_vector(doc)
        vector_A = doc_embedding
        #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
        terminos_candidatos = []
        for termino in catalog:
          vector_B = termino["embedding"]
          if similarity == 0:
            dist = numpy.linalg.norm(vector_A-vector_B)
            if dist <= similarity_threshold:
              #print(doc, " ", termino["term"], " Similarity:", dist)
              terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
            #Nos quedamos con el término con menor distancia de los obtenidos
            if terminos_candidatos:
              terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=False) 
              terminos_vinculados.append(terminos_candidatos_sorted[0])
          else:
            dist = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
            if dist >= similarity_threshold:
              #print(doc, " ", termino["term"], " Similarity:", dist)
              terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "similarity": dist})
            #Nos quedamos con el término con mayor probabilidad de los obtenidos
            if terminos_candidatos:
              terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('similarity'), reverse=True) 
              terminos_vinculados.append(terminos_candidatos_sorted[0])

  print("Resultado:")

  if similarity == 0:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "100.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]>termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  else:
    #Eliminamos duplicados
    terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
    termino_ant = {"term":"", "similarity": "0.0"}
    terminos_vinculados_deduplicated = []
    for termino in terminos_vinculados_sorted:
      if termino["term"] != termino_ant["term"]:
        if termino_ant["term"] != "":
          terminos_vinculados_deduplicated.append(termino_ant)
          #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])
        termino_ant = termino
      else:
        if termino_ant["similarity"]<termino["similarity"]:
          termino_ant = termino
    terminos_vinculados_deduplicated.append(termino_ant)
    #print(termino_ant["term"], " Cosine Similarity:", termino_ant["similarity"])

  return terminos_vinculados_deduplicated


In [ ]:
#Tiempo de ejecucion: 7 sg
terminos_vinculados = get_terminos_vinculados(model_cc50, terminos_snomedct_problemassaludAH_cc50, 3.6, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])


Resultado:
apendicitis  Similarity: 0.0
ascitis  Similarity: 0.0
cicatriz  Similarity: 0.0
cálculo renal  Similarity: 0.0
derrame pleural  Similarity: 0.0
hernia paraumbilical  Similarity: 2.389089627534487
hidronefrosis  Similarity: 0.0
hidroureteronefrosis  Similarity: 2.799965914540009
linfadenopatía  Similarity: 0.0
masa suprarrenal  Similarity: 0.0
obstrucción del intestino delgado  Similarity: 0.0


In [ ]:
#Tiempo de ejecucion: 14 sg
terminos_vinculados = get_terminos_vinculados(model_cc50, terminos_snomedct_problemassaludAH_cc50, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
apendicitis  Cosine Similarity: 0.999999936480693
ascitis  Cosine Similarity: 1.0000000269859675
aterosclerosis de aorta  Cosine Similarity: 0.9591700775957654
cicatriz  Cosine Similarity: 1.0000000107440326
cálculo renal  Cosine Similarity: 1.000000012533459
derrame pleural  Cosine Similarity: 0.9999999984069964
dolor de la pared abdominal  Cosine Similarity: 0.9547481720396018
estrechez del conducto biliar  Cosine Similarity: 0.967123320276856
hernia paraumbilical  Cosine Similarity: 0.9831991762383055
hidronefrosis  Cosine Similarity: 0.9999999838711423
hidroureteronefrosis  Cosine Similarity: 0.9787684673258287
lesión hepática  Cosine Similarity: 0.9572028197015432
linfadenopatía  Cosine Similarity: 1.000000041617977
masa abdominal  Cosine Similarity: 0.9683426464425468
masa pélvica  Cosine Similarity: 0.9639209134184615
masa suprarrenal  Cosine Similarity: 1.0000000411666718
miopía alta progresiva degenerativa  Cosine Similarity: 0.9502464248194111
obstrucción del intes

In [ ]:
#Tiempo de ejecucion: 7 sg
terminos_vinculados = get_terminos_vinculados(model_cc100, terminos_snomedct_problemassaludAH_cc100, 3.6, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Euclidean Similarity:", termino["similarity"])

Resultado:
apendicitis  Euclidean Similarity: 0.0
ascitis  Euclidean Similarity: 0.0
cicatriz  Euclidean Similarity: 0.0
cálculo renal  Euclidean Similarity: 0.0
derrame pleural  Euclidean Similarity: 0.0
hernia paraumbilical  Euclidean Similarity: 2.492255784624611
hidronefrosis  Euclidean Similarity: 0.0
hidroureteronefrosis  Euclidean Similarity: 3.0515556116232316
linfadenopatía  Euclidean Similarity: 0.0
masa suprarrenal  Euclidean Similarity: 0.0
obstrucción del intestino delgado  Euclidean Similarity: 0.0


In [ ]:
#Tiempo de ejecucion: 14 sg
terminos_vinculados = get_terminos_vinculados(model_cc100, terminos_snomedct_problemassaludAH_cc100, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
apendicitis  Cosine Similarity: 1.0000000330027676
ascitis  Cosine Similarity: 0.9999999817716361
aterosclerosis de aorta  Cosine Similarity: 0.9540795972645387
cicatriz  Cosine Similarity: 0.999999986298762
cálculo renal  Cosine Similarity: 0.9999999723004029
derrame pleural  Cosine Similarity: 1.0000000237612028
hernia paraumbilical  Cosine Similarity: 0.9829142065456283
hidronefrosis  Cosine Similarity: 0.9999999883575863
hidroureteronefrosis  Cosine Similarity: 0.9740628994661286
linfadenopatía  Cosine Similarity: 0.9999999508360591
masa abdominal  Cosine Similarity: 0.9515683032409354
masa suprarrenal  Cosine Similarity: 1.0000000343593185
obstrucción del intestino delgado  Cosine Similarity: 1.0000000083922036


In [ ]:
#Tiempo de ejecucion: 7 sg
terminos_vinculados = get_terminos_vinculados(model_cc300, terminos_snomedct_problemassaludAH_cc300, 3.6, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Euclidean Similarity:", termino["similarity"])

Resultado:
apendicitis  Euclidean Similarity: 0.0
ascitis  Euclidean Similarity: 0.0
cicatriz  Euclidean Similarity: 0.0
cálculo renal  Euclidean Similarity: 0.0
derrame pleural  Euclidean Similarity: 0.0
hernia paraumbilical  Euclidean Similarity: 2.6722831904510875
hidronefrosis  Euclidean Similarity: 0.0
hidroureteronefrosis  Euclidean Similarity: 3.21093541660065
linfadenopatía  Euclidean Similarity: 0.0
masa suprarrenal  Euclidean Similarity: 0.0
obstrucción del intestino delgado  Euclidean Similarity: 0.0


In [ ]:
#Tiempo de ejecucion: 14 sg
terminos_vinculados = get_terminos_vinculados(model_cc300, terminos_snomedct_problemassaludAH_cc300, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Cosine Similarity:", termino["similarity"])

Resultado:
apendicitis  Cosine Similarity: 0.9999999326309524
ascitis  Cosine Similarity: 0.9999999409117064
cicatriz  Cosine Similarity: 1.0000000310196664
cálculo renal  Cosine Similarity: 0.9999999895197529
derrame pleural  Cosine Similarity: 1.0000000329506975
hernia paraumbilical  Cosine Similarity: 0.9803130275701475
hidronefrosis  Cosine Similarity: 1.0000000501487158
hidroureteronefrosis  Cosine Similarity: 0.9738593477678313
linfadenopatía  Cosine Similarity: 1.0000000685270494
masa suprarrenal  Cosine Similarity: 1.0000000090424441
obstrucción del intestino delgado  Cosine Similarity: 1.000000050942657


In [ ]:
#informe ="el paciente presenta el abdomen distendido"
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."

In [ ]:
#Tiempo de ejecucion: 9 sg
terminos_vinculados = get_terminos_vinculados(model_cs50, terminos_snomedct_problemassaludAH_cs50, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
absceso abdominal  Similarity: 1.1068349166363272
adherencias intestinales con obstrucción  Similarity: 1.004833075831601
amplitud de movimiento articular activo reducido  Similarity: 1.167941177059055
anemia por pérdida crónica de sangre  Similarity: 1.1973448265257902
anomalía fetal sospechada que afecta el manejo de la madre  Similarity: 1.1854451817800258
apendicitis  Similarity: 0.0
artrosis primaria localizada, de la región pélvica y muslo  Similarity: 1.130894289716902
ascitis  Similarity: 0.0
aterosclerosis de injerto-puente de miembro  Similarity: 1.1686517344078948
atrofia sistémica múltiple  Similarity: 1.1737567867300949
calcificación de mama  Similarity: 0.9740240720534186
callo - lesión  Similarity: 1.1915649321707833
carcinoma in situ de parénquima pulmonar y bronquio del lóbulo superior  Similarity: 0.9829802036543631
cicatriz  Similarity: 0.0
cuadriplejía incompleta debida a lesión de médula espinal entre primera y cuarta vértebras cervicales  Similarity: 1.

In [ ]:
#Tiempo de ejecucion: 20 sg
terminos_vinculados = get_terminos_vinculados(model_cs50, terminos_snomedct_problemassaludAH_cs50, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
apendicitis  Similarity: 0.9999999356723795
ascitis  Similarity: 1.000000019192385
aterosclerosis de aorta  Similarity: 0.9531262237422627
cicatriz  Similarity: 1.0000000233752089
cálculo renal  Similarity: 1.0000000188722205
derrame pleural  Similarity: 0.9999999975373195
hernia paraumbilical  Similarity: 0.9723111344676618
hidronefrosis  Similarity: 1.0000000309150467
hidroureteronefrosis  Similarity: 0.9755432316003292
linfadenopatía  Similarity: 1.0000000109378062
masa suprarrenal  Similarity: 1.0000000165937915
obstrucción del intestino delgado  Similarity: 0.9999999459173821


In [ ]:
terminos_vinculados = get_terminos_vinculados(model_cs100, terminos_snomedct_problemassaludAH_cs100, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
apendicitis  Similarity: 0.0
ascitis  Similarity: 0.0
calcificación de mama  Similarity: 1.1559857123042463
cicatriz  Similarity: 0.0
cálculo renal  Similarity: 0.0
derrame pleural  Similarity: 0.0
enfermedad de Crohn de intestinos delgado Y grueso  Similarity: 1.119336685957993
estenosis degenerativa del conducto raquídeo de la región lumbar  Similarity: 1.098937152459598
estrechez del conducto biliar  Similarity: 1.1526721225573981
hernia paraumbilical  Similarity: 1.042304288474881
hidronefrosis  Similarity: 0.0
hidroureteronefrosis  Similarity: 1.143493328416541
linfadenopatía  Similarity: 0.0
masa suprarrenal  Similarity: 0.0
miopía alta progresiva degenerativa  Similarity: 1.1657013958435916
obstrucción del intestino delgado  Similarity: 0.0
pruebas de función hepática anormales  Similarity: 1.1861172749621762


In [ ]:
terminos_vinculados = get_terminos_vinculados(model_cs100, terminos_snomedct_problemassaludAH_cs100, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
apendicitis  Similarity: 0.9999999802842179
ascitis  Similarity: 0.9999999626764693
aterosclerosis de aorta  Similarity: 0.9529418925156002
cicatriz  Similarity: 0.9999999845765345
cálculo renal  Similarity: 1.00000004624113
derrame pleural  Similarity: 1.0000000018148436
hernia paraumbilical  Similarity: 0.9650706149281232
hidronefrosis  Similarity: 1.0000000356641567
hidroureteronefrosis  Similarity: 0.9627810207381385
linfadenopatía  Similarity: 1.000000029915514
masa suprarrenal  Similarity: 0.9999999547462615
obstrucción del intestino delgado  Similarity: 1.0000000207570674


In [ ]:
terminos_vinculados = get_terminos_vinculados(model_cs300, terminos_snomedct_problemassaludAH_cs300, 1.2, 0, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
apendicitis  Similarity: 0.0
ascitis  Similarity: 0.0
cicatriz  Similarity: 0.0
cálculo renal  Similarity: 0.0
derrame pleural  Similarity: 0.0
hernia paraumbilical  Similarity: 1.1935243176327452
hidronefrosis  Similarity: 0.0
linfadenopatía  Similarity: 0.0
masa suprarrenal  Similarity: 0.0
obstrucción del intestino delgado  Similarity: 0.0


In [ ]:
terminos_vinculados = get_terminos_vinculados(model_cs300, terminos_snomedct_problemassaludAH_cs300, 0.95, 1, informe )
for termino in terminos_vinculados:
  print(termino["term"], " Similarity:", termino["similarity"])

Resultado:
apendicitis  Similarity: 0.9999999962064328
ascitis  Similarity: 1.0000000085561287
cicatriz  Similarity: 1.0000000027894624
cálculo renal  Similarity: 0.9999999935315248
derrame pleural  Similarity: 0.9999999602383638
hernia paraumbilical  Similarity: 0.9617149205651632
hidronefrosis  Similarity: 1.0000000144906906
hidroureteronefrosis  Similarity: 0.9591198345534763
linfadenopatía  Similarity: 0.9999999615653624
masa suprarrenal  Similarity: 0.9999999898767472
obstrucción del intestino delgado  Similarity: 1.0000000646101876
